# StoryLand AI - Complete Feature Showcase

**Transform books into travel adventures using Google Agent Development Kit (ADK)**

This notebook demonstrates a production-ready multi-agent system that showcases:

## 🎯 Features Demonstrated

### Multi-Agent Architecture
- ✅ **Sequential Agents** - Metadata → Discovery → Composition workflow
- ✅ **Parallel Agents** - Concurrent city, landmark, and author discovery
- ✅ **LLM-Powered** - All agents powered by Gemini 2.0 Flash
- ✅ **Three-Phase Workflow** - Metadata extraction, discovery with region analysis, and composition

### Tools & Integration
- ✅ **Custom Tools** - Google Books API integration, user preferences tool
- ✅ **Built-in Tools** - Google Search for research
- ✅ **Pydantic Models** - Type-safe agent communication

### Sessions & State Management
- ✅ **InMemorySessionService** - Fast development and testing
- ✅ **DatabaseSessionService** - SQLite persistence across restarts
- ✅ **Multi-Scoped State** - Session, user, app, and temporary scopes
- ✅ **User Preferences** - Persistent across sessions with `user:` prefix

### Context Engineering
- ✅ **ContextManager** - Automatic context size monitoring
- ✅ **Sliding Window** - Keep recent events when context grows too large
- ✅ **Token Tracking** - Estimate and limit token usage

### Observability
- ✅ **Structured Logging** - Detailed execution logs with structlog
- ✅ **ADK LoggingPlugin** - Built-in agent observability
- ✅ **Event Tracking** - Monitor agent execution flow

### Evaluation
- ✅ **ADK Eval** - Automated rubric-based evaluation
- ✅ **Custom Rubrics** - Book relevance, preference adherence, completeness, etc.

### Advanced Features
- ✅ **Human-in-the-Loop** - Region selection for practical travel planning
- ✅ **Geographic Grouping** - LLM-based region analysis
- ✅ **Multi-User Support** - Isolated sessions and preferences per user

---

## 📚 What You'll Build

By the end of this notebook, you'll have executed a complete literary travel planning system that:
1. Extracts exact book metadata from Google Books API
2. Discovers cities, landmarks, and author-related sites
3. Groups locations into practical travel regions
4. Lets you select which regions to explore (HITL)
5. Creates a personalized itinerary based on your preferences

All with full observability, persistent sessions, and type-safe communication!

## 🔧 Setup & Configuration

In [84]:
# Standard library imports
import os
import sys
import json
import uuid
import logging
from datetime import datetime
from dotenv import load_dotenv

# Google ADK imports
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.plugins.logging_plugin import LoggingPlugin

# StoryLand AI imports
from services.session_service import create_session_service
from services.context_manager import ContextManager
from tools.google_books import google_books_tool
from agents.orchestrator import (
    create_metadata_stage,
    create_discovery_workflow,
    create_composition_workflow,
    create_eval_workflow,
)
from common.logging import configure_logging, get_logger

# Load environment
load_dotenv()

# Configure logging
configure_logging(level="INFO")
logger = get_logger(__name__)

print("✅ Environment Setup Complete")
print(f"   Google API Key: {'✓ Found' if os.getenv('GOOGLE_API_KEY') else '✗ MISSING'}")
print(f"   Python: {sys.version.split()[0]}")
print(f"   Logging: INFO level")

✅ Environment Setup Complete
   Google API Key: ✓ Found
   Python: 3.12.6
   Logging: INFO level


## 🎨 Section 1: Pydantic Data Models

StoryLand AI uses **Pydantic models** for type-safe agent communication. Each agent's output is validated against a schema, ensuring data quality and catching errors early.

### Benefits:
- **Type Safety** - Automatic validation of all fields
- **Clear Contracts** - Each agent knows exactly what to produce
- **Better Debugging** - Detailed validation error messages
- **IDE Support** - Autocomplete and type hints

In [85]:
from models.book import BookMetadata, BookContext
from models.discovery import (
    CityDiscovery, 
    LandmarkDiscovery, 
    AuthorSites,
    RegionAnalysis,  # NEW: Region grouping
)
from models.itinerary import TripItinerary, CityPlan, CityStop
from models.preferences import TravelPreferences

print("📦 Pydantic Models Loaded:")
print("\n   Book Models:")
print("     - BookMetadata (title, author, description, etc.)")
print("     - BookContext (locations, time_period, themes)")
print("\n   Discovery Models:")
print("     - CityDiscovery (list of cities with relevance)")
print("     - LandmarkDiscovery (list of landmarks with connections)")
print("     - AuthorSites (museums, birthplaces, etc.)")
print("     - RegionAnalysis (geographic grouping of cities) ⭐ NEW")
print("\n   Itinerary Models:")
print("     - TripItinerary (cities list + summary)")
print("     - CityPlan (city + days + stops)")
print("     - CityStop (name, type, reason, time_of_day, notes)")
print("\n   Preferences Model:")
print("     - TravelPreferences (budget, pace, museums, kids, etc.)")

# Show an example schema
print("\n📋 Example: CityStop Schema")
print(json.dumps(CityStop.model_json_schema(), indent=2)[:500] + "...")

📦 Pydantic Models Loaded:

   Book Models:
     - BookMetadata (title, author, description, etc.)
     - BookContext (locations, time_period, themes)

   Discovery Models:
     - CityDiscovery (list of cities with relevance)
     - LandmarkDiscovery (list of landmarks with connections)
     - AuthorSites (museums, birthplaces, etc.)
     - RegionAnalysis (geographic grouping of cities) ⭐ NEW

   Itinerary Models:
     - TripItinerary (cities list + summary)
     - CityPlan (city + days + stops)
     - CityStop (name, type, reason, time_of_day, notes)

   Preferences Model:
     - TravelPreferences (budget, pace, museums, kids, etc.)

📋 Example: CityStop Schema
{
  "description": "A stop/place to visit in a city",
  "properties": {
    "name": {
      "description": "Name of the place",
      "title": "Name",
      "type": "string"
    },
    "type": {
      "description": "Type: landmark, museum, cafe, restaurant, bookstore, etc.",
      "title": "Type",
      "type": "string"
    },
 

## 🤖 Section 2: Custom Tools

StoryLand AI uses custom tools to integrate with external APIs:

### 1. Google Books Tool
Searches Google Books API and returns Pydantic-validated book metadata.

### 2. Preferences Tool
Reads user preferences from session state (using `ToolContext.state`).

In [86]:
from tools.google_books import search_book
from tools.preferences import get_user_preferences

print("🔧 Custom Tools:")
print("\n   1. google_books_tool")
print("      Function: search_book(title, author)")
print("      Returns: BookMetadata JSON")
print("      Purpose: Search Google Books API")

print("\n   2. get_preferences_tool")
print("      Function: get_preferences()")
print("      Returns: User preferences from session state")
print("      Purpose: Access user:preferences via ToolContext")

# Demonstrate google_books_tool
print("\n📚 Demo: Search for 'Pride and Prejudice'")
try:
    result_json = search_book(title="Pride and Prejudice", author="Jane Austen")
    result = json.loads(result_json)
    print(f"\n   ✅ Found: {result.get('book_title')}")
    print(f"      Author: {result.get('author')}")
    print(f"      Published: {result.get('published_date')}")
    print(f"      Categories: {', '.join(result.get('categories', [])[:3])}")
except Exception as e:
    print(f"\n   ⚠️  Error: {e}")
    print("      (This is expected if no internet connection)")

🔧 Custom Tools:

   1. google_books_tool
      Function: search_book(title, author)
      Returns: BookMetadata JSON
      Purpose: Search Google Books API

   2. get_preferences_tool
      Function: get_preferences()
      Returns: User preferences from session state
      Purpose: Access user:preferences via ToolContext

📚 Demo: Search for 'Pride and Prejudice'
2025-11-25 21:44:27 [info     ] search_book_called             author='Jane Austen' title='Pride and Prejudice'
2025-11-25 21:44:27 [info     ] google_books_search            author='Jane Austen' title='Pride and Prejudice'
2025-11-25 21:44:27 [info     ] google_books_results           count=5
2025-11-25 21:44:27 [info     ] google_books_selected          author='Jane Austen' title='Pride and Prejudice'

   ✅ Found: Pride and Prejudice
      Author: Jane Austen
      Published: 1918
      Categories: Courtship


## 🏗️ Section 3: Multi-Agent Architecture

StoryLand AI uses a **three-phase workflow** with multiple agent types:

### Phase 1: Metadata Stage
- Extracts exact book title and author from Google Books API
- **Why?** Books like "The Nightingale" can match multiple books

### Phase 2: Discovery Workflow
- **Sequential agents**: book_context → reader_profile
- **Parallel agents**: city_discovery, landmark_discovery, author_sites (concurrent)
- **Region analyzer**: Groups cities into practical travel regions

### Phase 3: Composition Workflow
- Creates itinerary for user-selected region(s)
- Personalized based on preferences

### Agent Types:
- **SequentialAgent** - Executes sub-agents in order
- **ParallelAgent** - Runs sub-agents concurrently
- **LlmAgent** - Single LLM-powered agent with tools/output_schema

In [87]:
# Configure Gemini model
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

model = Gemini(
    model=os.getenv("MODEL_NAME", "gemini-2.0-flash-lite"),
    api_key=os.getenv("GOOGLE_API_KEY"),
    retry_options=retry_config,
)

print(f"🤖 Model Configured: {model.model}")
print(f"   Retry attempts: 5")
print(f"   Exponential backoff: enabled")

# Create the three-phase workflow stages
print("\n🏗️  Creating Three-Phase Workflow:")

# Phase 1: Metadata extraction
metadata_stage = create_metadata_stage(model, google_books_tool)
print("\n   ✅ Phase 1: Metadata Stage")
print("      - Searches Google Books API")
print("      - Extracts exact title and author")
print("      - Saves to state['book_metadata']")

# Phase 2: Discovery (will be created per-book)
print("\n   ✅ Phase 2: Discovery Workflow (created per book)")
print("      - book_context_pipeline (researches setting)")
print("      - reader_profile_agent (loads preferences)")
print("      - ParallelAgent (city, landmark, author) ⚡ concurrent")
print("      - region_analyzer_agent (groups cities by geography)")
print("      - Saves to state['region_analysis']")

# Phase 3: Composition (created separately)
print("\n   ✅ Phase 3: Composition Workflow")
print("      - trip_composer_agent (creates itinerary)")
print("      - Uses selected region(s) from HITL")
print("      - Saves to state['final_itinerary']")

# Also show eval workflow for reference
print("\n   📊 Bonus: Eval Workflow (for automated testing)")
print("      - Combines all 6 stages")
print("      - Auto-selects all regions (no HITL)")
print("      - Used for ADK eval with rubrics")

🤖 Model Configured: gemini-2.5-flash
   Retry attempts: 5
   Exponential backoff: enabled

🏗️  Creating Three-Phase Workflow:

   ✅ Phase 1: Metadata Stage
      - Searches Google Books API
      - Extracts exact title and author
      - Saves to state['book_metadata']

   ✅ Phase 2: Discovery Workflow (created per book)
      - book_context_pipeline (researches setting)
      - reader_profile_agent (loads preferences)
      - ParallelAgent (city, landmark, author) ⚡ concurrent
      - region_analyzer_agent (groups cities by geography)
      - Saves to state['region_analysis']

   ✅ Phase 3: Composition Workflow
      - trip_composer_agent (creates itinerary)
      - Uses selected region(s) from HITL
      - Saves to state['final_itinerary']

   📊 Bonus: Eval Workflow (for automated testing)
      - Combines all 6 stages
      - Auto-selects all regions (no HITL)
      - Used for ADK eval with rubrics


## 💾 Section 4: Sessions & State Management

ADK provides flexible session management:

### InMemorySessionService
- Fast, perfect for development
- Sessions lost on restart

### DatabaseSessionService (SQLite)
- Persistent across restarts
- Enables analytics and user profiling

### State Scopes:
- **Session scope** (`state["key"]`) - Per conversation
- **User scope** (`state["user:key"]`) - Persists across all user sessions
- **App scope** (`state["app:key"]`) - Global for all users
- **Temporary scope** (`state["temp:key"]`) - Never persisted

In [88]:
# Choose session service type
USE_DATABASE = True  # Set to False for in-memory only

session_service = create_session_service(use_database=USE_DATABASE)

print(f"💾 Session Service: {'DatabaseSessionService (SQLite)' if USE_DATABASE else 'InMemorySessionService'}")
if USE_DATABASE:
    print("   Database: storyland_sessions.db")
    print("   ✅ Sessions persist across restarts")
    print("   ✅ User preferences persist")
    print("   ✅ Can query with SQL")
else:
    print("   ⚠️  Sessions lost on restart")
    print("   ✅ Fast for development")

print("\n📊 State Scopes:")
print("   - state['key']         → Session scope (per conversation)")
print("   - state['user:key']    → User scope (persists across sessions)")
print("   - state['app:key']     → App scope (global for all users)")
print("   - state['temp:key']    → Temporary (never persisted)")

Using DatabaseSessionService with: sqlite:///storyland_sessions.db
2025-11-25 16:44:27 - google_adk.google.adk.sessions.database_session_service - INFO - Local timezone: America/New_York
💾 Session Service: DatabaseSessionService (SQLite)
   Database: storyland_sessions.db
   ✅ Sessions persist across restarts
   ✅ User preferences persist
   ✅ Can query with SQL

📊 State Scopes:
   - state['key']         → Session scope (per conversation)
   - state['user:key']    → User scope (persists across sessions)
   - state['app:key']     → App scope (global for all users)
   - state['temp:key']    → Temporary (never persisted)


## 👤 Section 5: User Preferences & Personalization

User preferences are stored in `state["user:preferences"]` and persist across sessions.

### How it works:
1. User preferences set in session state
2. `reader_profile_agent` calls `get_preferences_tool`
3. Tool reads from `ToolContext.state["user:preferences"]`
4. `trip_composer_agent` sees preferences summary
5. Itinerary is personalized

In [89]:
# Define user preferences
user_id = "demo_user"
session_id = str(uuid.uuid4())

user_preferences = {
    "budget": "moderate",           # budget, moderate, luxury
    "preferred_pace": "relaxed",    # relaxed, moderate, fast-paced
    "prefers_museums": True,
    "travels_with_kids": False,
    "dietary_restrictions": ["vegetarian"],
    "favorite_genres": ["historical fiction", "classics"],
}

print("👤 User Preferences:")
print(json.dumps(user_preferences, indent=2))
print("\n   These will be stored as state['user:preferences']")
print("   They persist across all sessions for this user!")

👤 User Preferences:
{
  "budget": "moderate",
  "preferred_pace": "relaxed",
  "prefers_museums": true,
  "travels_with_kids": false,
  "dietary_restrictions": [
    "vegetarian"
  ],
  "favorite_genres": [
    "historical fiction",
    "classics"
  ]
}

   These will be stored as state['user:preferences']
   They persist across all sessions for this user!


## 🎯 Section 6: Context Engineering

The `ContextManager` monitors conversation size and applies compaction when needed.

### Features:
- **Token Estimation** - Approximate token count from text length
- **Event Limiting** - Keep only recent N events (sliding window)
- **Compaction Detection** - Automatically detect when to compact
- **System Prompt Preservation** - Keep important system messages

In [90]:
# Create context manager with limits
context_manager = ContextManager(
    max_events=20,        # Keep max 20 events
    max_tokens=30000,     # Approximate token limit
    preserve_system=True  # Always keep system prompts
)

print("🎯 Context Manager Configuration:")
print(f"   Max events: {context_manager.max_events}")
print(f"   Max tokens: {context_manager.max_tokens:,}")
print(f"   Preserve system prompts: {context_manager.preserve_system}")

# Demo: Simulate a long conversation
from google.genai import types

class MockEvent:
    def __init__(self, text):
        self.content = types.Content(parts=[types.Part(text=text)])

# Create mock conversation
mock_events = [MockEvent(f"Event {i}: {'x' * 100}") for i in range(25)]

# Get statistics
stats = context_manager.get_context_stats(mock_events)
print(f"\n📊 Example Conversation:")
print(f"   Total events: {stats['num_events']}")
print(f"   Total characters: {stats['total_chars']:,}")
print(f"   Estimated tokens: ~{stats['estimated_tokens']}")
print(f"   Within limits: {stats['within_limit']}")

# Show compaction
if context_manager.should_compact(mock_events):
    compacted = context_manager.limit_events(mock_events, num_recent=15)
    after_stats = context_manager.get_context_stats(compacted)
    
    print(f"\n🗜️  After Compaction:")
    print(f"   Events: {len(mock_events)} → {len(compacted)}")
    print(f"   Tokens: ~{stats['estimated_tokens']} → ~{after_stats['estimated_tokens']}")
    print(f"   Saved: ~{stats['estimated_tokens'] - after_stats['estimated_tokens']} tokens")

🎯 Context Manager Configuration:
   Max events: 20
   Max tokens: 30,000
   Preserve system prompts: True

📊 Example Conversation:
   Total events: 25
   Total characters: 2,740
   Estimated tokens: ~685
   Within limits: False

🗜️  After Compaction:
   Events: 25 → 15
   Tokens: ~685 → ~412
   Saved: ~273 tokens


## 📝 Section 7: Observability - Logging

StoryLand AI uses multiple logging layers:

### 1. Structured Logging (structlog)
- Key-value logging for easy parsing
- Defined in `common/logging.py`

### 2. ADK LoggingPlugin
- Built-in agent observability
- Tracks agent execution, tool calls, etc.

### 3. Event Monitoring
- Track each agent's output via `runner.run_async()`

In [91]:
from common.logging import get_logger

# Get a structured logger
demo_logger = get_logger("showcase_demo")

print("📝 Logging Configuration:")
print("\n   1. Structured Logging (structlog)")
demo_logger.info("example_event", user_id="demo_user", action="showcase", count=42)
print("      ↑ Notice: key=value format")

print("\n   2. ADK LoggingPlugin")
print("      - Automatically logs agent execution")
print("      - Tool calls and responses")
print("      - Model interactions")

print("\n   3. Event Monitoring")
print("      - Track via runner.run_async()")
print("      - Shows agent names, tool calls, responses")
print("      - Detect final responses with event.is_final_response()")

📝 Logging Configuration:

   1. Structured Logging (structlog)
2025-11-25 21:44:27 [info     ] example_event                  action=showcase count=42 user_id=demo_user
      ↑ Notice: key=value format

   2. ADK LoggingPlugin
      - Automatically logs agent execution
      - Tool calls and responses
      - Model interactions

   3. Event Monitoring
      - Track via runner.run_async()
      - Shows agent names, tool calls, responses
      - Detect final responses with event.is_final_response()


## 📊 Section 8: Agent Evaluation

StoryLand AI uses **ADK Eval** for automated testing with rubric-based scoring.

### Evaluation Rubrics:
- `book_relevance` - Locations connected to book
- `preference_adherence` - Respects user preferences
- `completeness` - Comprehensive itinerary
- `actionability` - Practical details
- `geographical_accuracy` - Real places, correct countries

### Running Evals:
```bash
.venv/bin/adk eval agents/storyland single_test \
  --config_file_path tests/evaluation/eval_config.json \
  --print_detailed_results
```

In [92]:
import os
from pathlib import Path

# Check if eval config exists
eval_config_path = Path("tests/evaluation/eval_config.json")

print("📊 Agent Evaluation:")
print(f"\n   Config file: {eval_config_path}")
print(f"   Exists: {eval_config_path.exists()}")

if eval_config_path.exists():
    with open(eval_config_path) as f:
        eval_config = json.load(f)
    
    print(f"\n   Rubrics defined: {len(eval_config.get('rubrics', []))}")
    print("\n   Rubric Details:")
    for rubric in eval_config.get('rubrics', []):
        print(f"     - {rubric.get('name')}: {rubric.get('description', '')[:60]}...")

print("\n   💡 Run evals with:")
print("      .venv/bin/adk eval agents/storyland single_test \\")
print("        --config_file_path tests/evaluation/eval_config.json \\")
print("        --print_detailed_results")

print("\n   🎯 Uses create_eval_workflow() which auto-selects all regions")

📊 Agent Evaluation:

   Config file: tests/evaluation/eval_config.json
   Exists: True

   Rubrics defined: 0

   Rubric Details:

   💡 Run evals with:
      .venv/bin/adk eval agents/storyland single_test \
        --config_file_path tests/evaluation/eval_config.json \
        --print_detailed_results

   🎯 Uses create_eval_workflow() which auto-selects all regions


## 🚀 Section 9: Full End-to-End Demo

Now let's run the complete three-phase workflow:

1. **Phase 1**: Extract exact book metadata
2. **Phase 2**: Discover locations and analyze regions
3. **Human-in-the-Loop**: Select regions (simulated)
4. **Phase 3**: Create personalized itinerary

### Configure Your Book Here:

In [93]:
# ============================================================
# CONFIGURE YOUR BOOK AND PREFERENCES HERE
# ============================================================
BOOK_TITLE = "Pride and Prejudice"
AUTHOR = "Jane Austen"  # Optional

# User preferences (customize these!)
USER_PREFERENCES = {
    "budget": "moderate",
    "preferred_pace": "relaxed",
    "prefers_museums": True,
    "travels_with_kids": False,
}
# ============================================================

print(f"📚 Book: {BOOK_TITLE}")
print(f"   Author: {AUTHOR or 'Unknown'}")
print(f"\n👤 Preferences:")
for key, value in USER_PREFERENCES.items():
    print(f"   - {key}: {value}")

📚 Book: Pride and Prejudice
   Author: Jane Austen

👤 Preferences:
   - budget: moderate
   - preferred_pace: relaxed
   - prefers_museums: True
   - travels_with_kids: False


### Create Session with Preferences

In [94]:
# Create session
user_id = "demo_user"
session_id = str(uuid.uuid4())

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_id,
    state={
        "user:preferences": USER_PREFERENCES  # Persists across sessions!
    },
)

print(f"✅ Session Created")
print(f"   User ID: {user_id}")
print(f"   Session ID: {session_id[:8]}...")
print(f"   Preferences stored in state['user:preferences']")

✅ Session Created
   User ID: demo_user
   Session ID: d1545072...
   Preferences stored in state['user:preferences']


### Phase 1: Extract Book Metadata

In [95]:
print("="*70)
print("PHASE 1: EXTRACT BOOK METADATA")
print("="*70)

# Create runner for metadata stage
metadata_runner = Runner(
    agent=metadata_stage,
    app_name="storyland",
    session_service=session_service,
    plugins=[LoggingPlugin()],
)

metadata_prompt = f'Find book metadata for "{BOOK_TITLE}" by {AUTHOR or "unknown author"}.'
metadata_message = types.Content(role="user", parts=[types.Part(text=metadata_prompt)])

print(f"\n📝 Prompt: {metadata_prompt}")
print(f"\n🔄 Executing metadata extraction...\n")

event_count = 0
async with metadata_runner:
    async for event in metadata_runner.run_async(
        user_id=user_id, session_id=session_id, new_message=metadata_message
    ):
        event_count += 1
        if event.author:
            print(f"[{event_count}] {event.author}")

# Get metadata from session state
session = await session_service.get_session(
    app_name="storyland", user_id=user_id, session_id=session_id
)
book_metadata = session.state.get("book_metadata", {})
exact_title = book_metadata.get("book_title", BOOK_TITLE)
exact_author = book_metadata.get("author", AUTHOR or "Unknown")

print(f"\n✅ Metadata extracted!")
print(f"   Exact title: {exact_title}")
print(f"   Exact author: {exact_author}")
print(f"   Published: {book_metadata.get('published_date', 'N/A')}")

PHASE 1: EXTRACT BOOK METADATA
2025-11-25 16:44:28 - google_adk.google.adk.plugins.plugin_manager - INFO - Plugin 'logging_plugin' registered.
2025-11-25 16:44:28 - google_adk.google.adk.runners - WARNING - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".

📝 Prompt: Find book metadata for "Pride and Prejudice" by Jane Austen.

🔄 Executing metadata extraction...

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-43a3bf7d-ed12-4c00-abb2-32c2fc7acd73
[logging_plugin]    Session ID: d1545072-e8e2-4719-b94e-7a496081bcb4
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: storyland
[logging_plugin]    Root Agent: metadata_stage
[logging_plugin]    User Content: text: 'Find book metadata for "Pride and Prejudice" by Jane Austen.'
[logging_plugin] 🏃 INVO

### Phase 2: Discovery + Region Analysis

In [96]:
print("="*70)
print("PHASE 2: DISCOVERY + REGION ANALYSIS")
print("="*70)

# Create discovery workflow with exact book info
discovery_workflow = create_discovery_workflow(model, book_title=exact_title, author=exact_author)

discovery_runner = Runner(
    agent=discovery_workflow,
    app_name="storyland",
    session_service=session_service,
    plugins=[LoggingPlugin()],
)

discovery_prompt = f'Discover travel locations for "{exact_title}" by {exact_author}.\n\nFind cities, landmarks, and author-related sites, then group them into practical travel regions.'
discovery_message = types.Content(role="user", parts=[types.Part(text=discovery_prompt)])

print(f"\n📝 Prompt: Discover locations and group into regions")
print(f"\n🔄 Executing discovery workflow...\n")

async with discovery_runner:
    async for event in discovery_runner.run_async(
        user_id=user_id, session_id=session_id, new_message=discovery_message
    ):
        event_count += 1
        if event.author:
            print(f"[{event_count}] {event.author}")

# Get region analysis from session state
session = await session_service.get_session(
    app_name="storyland", user_id=user_id, session_id=session_id
)
region_analysis = session.state.get("region_analysis", {})

print(f"\n✅ Discovery complete!")
print(f"   Regions found: {len(region_analysis.get('regions', []))}")

PHASE 2: DISCOVERY + REGION ANALYSIS
2025-11-25 16:44:33 - google_adk.google.adk.plugins.plugin_manager - INFO - Plugin 'logging_plugin' registered.
2025-11-25 16:44:33 - google_adk.google.adk.runners - WARNING - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".

📝 Prompt: Discover locations and group into regions

🔄 Executing discovery workflow...

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-ce69d125-07d1-4152-8437-d5549cf753ca
[logging_plugin]    Session ID: d1545072-e8e2-4719-b94e-7a496081bcb4
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: storyland
[logging_plugin]    Root Agent: discovery_workflow
[logging_plugin]    User Content: text: 'Discover travel locations for "Pride and Prejudice" by Jane Austen.

Find cities, landmarks, an

### Human-in-the-Loop: Region Selection

Display region options and let the user select which to explore.

In [97]:
print("="*70)
print("TRAVEL REGION OPTIONS")
print("="*70)

regions = region_analysis.get("regions", [])

if not regions:
    print("\n⚠️  No regions found. Using all discovered cities.")
    selected_regions = []  # Will use all cities
else:
    print(f"\n{region_analysis.get('summary', 'Cities grouped by geographic proximity.')}\n")
    
    for i, region in enumerate(regions, 1):
        print(f"[{i}] {region.get('region_name')}")
        
        cities = region.get('cities', [])
        city_names = [c.get('name') for c in cities if c.get('name')]
        print(f"    Cities: {', '.join(city_names)}")
        
        if region.get('estimated_days'):
            print(f"    Duration: ~{region.get('estimated_days')} days")
        
        if region.get('travel_note'):
            print(f"    Travel: {region.get('travel_note')}")
        
        if region.get('highlights'):
            highlights = region.get('highlights')[:2] if isinstance(region.get('highlights'), list) else [region.get('highlights')]
            print(f"    Highlights: {', '.join(highlights)}")
        
        print()
    
    # For demo purposes, auto-select the first region
    # In a real notebook, you'd use: input("Which region(s)? ")
    print("📝 For this demo, auto-selecting region #1\n")
    selected_regions = [regions[0]]  # Select first region
    
    print(f"✅ Selected: {selected_regions[0].get('region_name')}")

# Validate that we have regions to work with
if not selected_regions:
    error_msg = (
        "No regions available to create an itinerary. "
        "The discovery phase did not find enough locations to group into travel regions. "
        "Try a different book or check the discovery results."
    )
    print(f"\n❌ Error: {error_msg}")
    raise ValueError(error_msg)

# Store selected regions in session state for trip_composer to access
session = await session_service.get_session(
    app_name="storyland", user_id=user_id, session_id=session_id
)
session.state["selected_regions"] = selected_regions

print(f"\n💾 Stored {len(selected_regions)} region(s) in session state")

TRAVEL REGION OPTIONS

Cities grouped by geographic proximity.

[1] Hampshire - Jane Austen's Heartlands
    Cities: Chawton, Steventon, Winchester
    Duration: ~3 days
    Travel: All locations are within Hampshire and easily accessible by car, with short drives (30-45 minutes) between cities.
    Highlights: Explore Jane Austen's actual home and writing place, her birthplace site, her family's manor, and her final resting place in Winchester Cathedral. This region offers the deepest personal connection to the author.

[2] North Midlands - Pemberley & Peak District
    Cities: Bakewell, Sudbury, Disley, Peak District National Park
    Duration: ~4 days
    Travel: Travel by car is recommended to navigate between the stately homes and scenic Peak District locations, with drives typically 1-1.5 hours.
    Highlights: Visit iconic 'Pemberley' film locations like Chatsworth House (2005 film) and Lyme Park (1995 BBC series), explore Sudbury Hall for interior shots, and experience the natu

### Phase 3: Create Personalized Itinerary

In [98]:
print("="*70)
print("PHASE 3: CREATE PERSONALIZED ITINERARY")
print("="*70)

# Create composition workflow
composition_workflow = create_composition_workflow(model)

composition_runner = Runner(
    agent=composition_workflow,
    app_name="storyland",
    session_service=session_service,
    plugins=[LoggingPlugin()],
)

composition_prompt = f"""Create a personalized literary travel itinerary for "{exact_title}" by {exact_author}.

Use ONLY the cities from the selected region(s): {json.dumps(selected_regions)}

Create a personalized itinerary based on user preferences and the selected region(s).
Include ALL cities from the selected regions in your itinerary."""

composition_message = types.Content(role="user", parts=[types.Part(text=composition_prompt)])

print(f"\n📝 Prompt: Create itinerary for selected region(s)")
print(f"\n🔄 Executing composition...\n")

final_response = None
async with composition_runner:
    async for event in composition_runner.run_async(
        user_id=user_id, session_id=session_id, new_message=composition_message
    ):
        event_count += 1
        if event.author:
            print(f"[{event_count}] {event.author}")
        if event.is_final_response():
            final_response = event

print(f"\n✅ Itinerary created! ({event_count} total events)")

PHASE 3: CREATE PERSONALIZED ITINERARY
2025-11-25 16:45:37 - google_adk.google.adk.plugins.plugin_manager - INFO - Plugin 'logging_plugin' registered.
2025-11-25 16:45:37 - google_adk.google.adk.runners - WARNING - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".

📝 Prompt: Create itinerary for selected region(s)

🔄 Executing composition...

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-0e304ba9-8eda-42b6-8e2c-bd6507c01e2d
[logging_plugin]    Session ID: d1545072-e8e2-4719-b94e-7a496081bcb4
[logging_plugin]    User ID: demo_user
[logging_plugin]    App Name: storyland
[logging_plugin]    Root Agent: composition_workflow
[logging_plugin]    User Content: text: 'Create a personalized literary travel itinerary for "Pride and Prejudice" by Jane Austen.

Use ONLY 

## 🎉 Section 10: Display Final Itinerary

Parse and display the complete travel plan.

In [99]:
# Get final itinerary from session state
session = await session_service.get_session(
    app_name="storyland", user_id=user_id, session_id=session_id
)
itinerary = session.state.get("final_itinerary", {})

print("="*70)
print(f"LITERARY TRAVEL ITINERARY")
print(f"{exact_title} by {exact_author}")
print("="*70)

# Display summary
if itinerary.get("summary_text"):
    print(f"\n📖 {itinerary['summary_text']}\n")

# Display cities
cities = itinerary.get("cities", [])
if cities:
    print(f"\n🌍 {len(cities)} {'City' if len(cities) == 1 else 'Cities'} to Visit:")
    print("="*70)
    
    for i, city in enumerate(cities, 1):
        print(f"\n{i}. {city.get('name')}, {city.get('country')}")
        print(f"   Days: {city.get('days_suggested')}")
        
        if city.get('overview'):
            print(f"   {city['overview']}")
        
        stops = city.get('stops', [])
        if stops:
            print(f"\n   📍 {len(stops)} Stops:")
            for j, stop in enumerate(stops, 1):
                print(f"\n      {j}. {stop.get('name')} [{stop.get('type')}]")
                print(f"         Time: {stop.get('time_of_day', 'any time').upper()}")
                print(f"         Why: {stop.get('reason')}")
                if stop.get('notes'):
                    print(f"         💡 {stop.get('notes')}")

    # Statistics
    total_days = sum(c.get('days_suggested', 0) for c in cities)
    total_stops = sum(len(c.get('stops', [])) for c in cities)
    
    print("\n" + "="*70)
    print("📊 TRIP STATISTICS")
    print("="*70)
    print(f"   Total cities: {len(cities)}")
    print(f"   Total days: {total_days}")
    print(f"   Total stops: {total_stops}")
    print(f"   Budget: {USER_PREFERENCES.get('budget', 'N/A')}")
    print(f"   Pace: {USER_PREFERENCES.get('preferred_pace', 'N/A')}")
else:
    print("\n⚠️  No itinerary data found.")

LITERARY TRAVEL ITINERARY
Pride and Prejudice by Jane Austen

📖 This personalized itinerary guides you through 'Hampshire - Jane Austen's Heartlands,' offering a relaxed and deeply immersive journey into the life and inspiration of Jane Austen. As a museum enthusiast with a moderate budget, you'll delve into the intimate details of her homes in Chawton and Steventon, culminating in a respectful visit to her final resting place in Winchester Cathedral. Expect leisurely days filled with literary discovery, perfectly paced for reflection and enjoyment of England's charming countryside.


🌍 3 Cities to Visit:

1. Chawton, England
   Days: 1
   Immerse yourself in the very home where Jane Austen lived and revised 'Pride and Prejudice.' Chawton offers an intimate glimpse into her daily life and the literary world she created.

   📍 3 Stops:

      1. Jane Austen's House Museum [museum]
         Time: FULL_DAY
         Why: This is Jane Austen's final home where she lived from 1809 until 1817

## 📊 Section 11: Session Statistics

Check session state and context usage.

In [100]:
# Get final session state
final_session = await session_service.get_session(
    app_name="storyland", user_id=user_id, session_id=session_id
)

print("📊 SESSION STATISTICS")
print("="*70)
print(f"   User ID: {user_id}")
print(f"   Session ID: {session_id[:8]}...")
print(f"   Total events: {len(final_session.events)}")

# Show state keys
state_keys = list(final_session.state.keys())
print(f"\n   State keys ({len(state_keys)}):")
for key in state_keys:
    if key.startswith("user:"):
        print(f"     - {key} (user-scoped, persists)")
    else:
        print(f"     - {key} (session-scoped)")

# Context statistics
stats = context_manager.get_context_stats(final_session.events)
print(f"\n   Context:")
print(f"     Total characters: {stats['total_chars']:,}")
print(f"     Estimated tokens: ~{stats['estimated_tokens']:,}")
print(f"     Within limit: {stats['within_limit']}")

📊 SESSION STATISTICS
   User ID: demo_user
   Session ID: d1545072...
   Total events: 20

   State keys (9):
     - book_metadata (session-scoped)
     - book_context (session-scoped)
     - reader_profile (session-scoped)
     - author_sites (session-scoped)
     - landmark_discovery (session-scoped)
     - city_discovery (session-scoped)
     - region_analysis (session-scoped)
     - final_itinerary (session-scoped)
     - user:preferences (user-scoped, persists)

   Context:
     Total characters: 31,261
     Estimated tokens: ~7,815
     Within limit: True


## 🎓 Summary

Congratulations! You've just executed a complete multi-agent literary travel planning system.

### ✅ Features You Experienced:

#### Multi-Agent Architecture
- Sequential agents (metadata → discovery → composition)
- Parallel agents (concurrent city/landmark/author discovery)
- Three-phase workflow with HITL region selection

#### Tools & Integration
- Custom tools (Google Books API, preferences)
- Built-in tools (Google Search)
- Pydantic models for type-safe communication

#### Sessions & State
- Database-backed sessions (SQLite)
- Multi-scoped state (session, user, app, temp)
- User preferences persisting across sessions

#### Context Engineering
- Context manager for size monitoring
- Token estimation and event limiting

#### Observability
- Structured logging (structlog)
- ADK LoggingPlugin
- Event tracking and monitoring

#### Evaluation
- ADK eval with custom rubrics
- Automated testing framework

---

## 🚀 Next Steps

### Try These:

1. **Change the book** - Edit `BOOK_TITLE` and `AUTHOR` above and re-run
2. **Customize preferences** - Modify `USER_PREFERENCES` (budget, pace, museums, kids)
3. **Select different regions** - Choose different region numbers when prompted
4. **Use the CLI** - Try `python main.py "Book Title" --budget luxury --pace relaxed`
5. **Run evaluations** - Execute ADK eval with `adk eval agents/storyland single_test`
6. **Query the database** - Use SQL to analyze user sessions and preferences
7. **Add new agents** - Create custom agents in `agents/` directory
8. **Add new tools** - Build custom tools in `tools/` directory

### Learn More:

- **Google ADK Docs**: https://ai.google.dev/adk
- **Project README**: [README.md](README.md)
- **Agent Code**: [agents/orchestrator.py](agents/orchestrator.py)
- **Models**: [models/](models/) directory
- **Tests**: [tests/unit/](tests/unit/) directory

---

## 📝 CLI Quick Reference

```bash
# Basic usage
python main.py "Pride and Prejudice"

# With preferences
python main.py "Gone with the Wind" --budget luxury --pace fast-paced

# With author
python main.py "1984" --author "George Orwell"

# With database persistence
python main.py "Harry Potter" --database

# Family trip
python main.py "Harry Potter" --with-kids --budget moderate

# Run tests
.venv/bin/pytest tests/unit/ -v

# Run evaluation
.venv/bin/adk eval agents/storyland single_test \
  --config_file_path tests/evaluation/eval_config.json

# ADK Web UI
.venv/bin/adk web agents/storyland
```

---

*Built with Google Agent Development Kit* 🤖